# Algorithm 27: Torsion Angle Loss

The Torsion Angle Loss supervises the predicted backbone and sidechain torsion angles (phi, psi, omega, chi1-4).

## Algorithm Pseudocode

![torsionAngleLoss](../imgs/algorithms/torsionAngleLoss.png)

## Source Code Location
- **File**: `AF2-source-code/model/folding.py`
- **Function**: `supervised_chi_loss`
- **Lines**: 480-550

## Torsion Angles in Proteins

| Angle | Description | Typical Range |
|-------|-------------|---------------|
| ω (omega) | Peptide bond | ~180° (trans) |
| φ (phi) | N-Cα bond | -180° to 180° |
| ψ (psi) | Cα-C bond | -180° to 180° |
| χ1-χ4 | Sidechain bonds | Rotameric |

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def torsion_angle_loss(pred_angles, gt_angles, angle_mask):
    """
    Torsion Angle Loss - Algorithm 27.
    
    Computes loss between predicted and ground truth torsion angles.
    Uses (sin, cos) representation for proper angular comparison.
    
    Args:
        pred_angles: Predicted angles as (sin, cos) [N_res, 7, 2]
        gt_angles: Ground truth angles as (sin, cos) [N_res, 7, 2]
        angle_mask: Valid angle mask [N_res, 7]
    
    Returns:
        Loss value
    """
    N_res = pred_angles.shape[0]
    
    print(f"Torsion Angle Loss")
    print(f"  Residues: {N_res}")
    print(f"  Angles per residue: 7 (omega, phi, psi, chi1-4)")
    
    # Normalize predictions to unit circle
    pred_norm = np.sqrt(np.sum(pred_angles**2, axis=-1, keepdims=True) + 1e-8)
    pred_normalized = pred_angles / pred_norm
    
    # Squared difference in (sin, cos) space
    # This correctly handles angular periodicity
    diff = pred_normalized - gt_angles
    sq_diff = np.sum(diff**2, axis=-1)  # [N_res, 7]
    
    # Apply mask
    masked_diff = sq_diff * angle_mask
    
    # Mean over valid angles
    n_valid = np.sum(angle_mask)
    loss = np.sum(masked_diff) / (n_valid + 1e-8)
    
    print(f"  Valid angles: {int(n_valid)}")
    print(f"  Loss: {loss:.4f}")
    
    return loss


def angle_to_sincos(angles):
    """Convert angles to (sin, cos) representation."""
    return np.stack([np.sin(angles), np.cos(angles)], axis=-1)

In [ ]:
# Test
N_res = 32

# Ground truth angles (radians)
gt_angles_raw = np.random.randn(N_res, 7) * 0.5
gt_angles = angle_to_sincos(gt_angles_raw)

# Predicted angles with some noise
pred_angles_raw = gt_angles_raw + np.random.randn(N_res, 7) * 0.1
pred_angles = angle_to_sincos(pred_angles_raw)

# Mask: all backbone angles valid, some chi angles masked
angle_mask = np.ones((N_res, 7))
angle_mask[:, 4:] = np.random.choice([0, 1], size=(N_res, 3))  # Random chi mask

print("Test Torsion Angle Loss")
print("="*50)

loss = torsion_angle_loss(pred_angles, gt_angles, angle_mask)

In [ ]:
# Test: Loss decreases as predictions improve
print("\nLoss vs prediction quality:")

for noise_level in [1.0, 0.5, 0.1, 0.01]:
    pred_noisy = gt_angles_raw + np.random.randn(N_res, 7) * noise_level
    pred = angle_to_sincos(pred_noisy)
    loss = torsion_angle_loss(pred, gt_angles, angle_mask)
    print(f"  Noise σ={noise_level}: Loss={loss:.4f}")

## Source Code Reference

```python
# From AF2-source-code/model/folding.py

def supervised_chi_loss(ret, batch, value):
  """Computes loss for torsion angles.

  Jumper et al. (2021) Suppl. Alg. 27 "torsionAngleLoss"
  """
  chi_mask = batch['chi_mask']
  pred_angles = ret['sidechains']['angles_sin_cos']
  
  # Normalize predictions
  pred_angles = pred_angles / jnp.linalg.norm(pred_angles, axis=-1, keepdims=True)
  
  # Compute angular loss
  sq_chi_error = jnp.sum(squared_difference(pred_angles, true_chi), axis=-1)
  sq_chi_loss = utils.mask_mean(mask=chi_mask, value=sq_chi_error)
  
  return sq_chi_loss
```